# Visualización de Tablas

En este notebook nos haremos preguntas sobre el _dataset_ Encuesta Origen-Destino de Santiago 2012, también conocida como EOD2012. Esta encuesta fue realizada para la [Secretaría de Transporte](https://www.sectra.gob.cl/biblioteca/detalle1.asp?mfn=3253).

## Preámbulo

In [ ]:
from pathlib import Path

EOD_PATH = Path("data") / "EOD_STGO"

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from chiricoca.config import setup_style

setup_style()

## Datos

La EOD2012 contiene varias base de datos. Cargaremos primero la de _personas_.

In [ ]:
import huedhued.eod_scl as eod

personas = eod.read_people(EOD_PATH)
personas

Noten que cada persona tiene un identificador único `Persona` y un identificador no único `Hogar`. 

Le agregaremos dos atributos a la tabla:

* `Edad`: atributo cuantitativo definido como el año 2013 menos el año de nacimiento.
* `GrupoEtareo`: atributo ordinal definido como grupos de edad de 5 años. Se calcula a partir de `Edad` con la operación módulo.

In [ ]:
personas["Edad"] = 2013 - personas["AnoNac"]

personas["GrupoEtareo"] = personas["Edad"] - (personas["Edad"] % 5)
personas["GrupoEtareo"].value_counts()

También leeremos la tabla de _hogares_. En esta tabla, cada fila representa a un hogar, esta vez con identificador `Hogar` único.

In [ ]:
hogares = eod.read_homes(EOD_PATH)
hogares

Finalmente cargamos la tabla de _viajes_, que tiene un viaje en cada fila. Cada `Viaje` es único, fue hecho por una `Persona`, que vive en un `Hogar`.

In [ ]:
viajes = eod.read_trips(EOD_PATH)
viajes

Los identificadores nos permiten cruzar las tres tablas y crear una sola, que llamaremos `tabla_completa`.

In [ ]:
tabla_completa = (viajes.merge(personas)).merge(hogares.drop("TipoDia", axis=1))

In [ ]:
tabla_completa.sample(5)

Utilizaremos esta tabla para la mayoría de los análisis en este notebook.

Sin embargo, antes de seguir nos preguntaremos: ¿a quién representa la encuesta? ¿Cómo entrevistar a un subconjunto de la población nos permite hacer análisis representativos de esta?

La respuesta está en los _factores de expansión_. Estos números se definen como:

> [...] la capacidad que tiene cada individuo seleccionado en una muestra probabilística para representar el universo en el cual esta contenido. Es decir, es la magnitud de representación que cada selección posee para describir una parte del universo de estudio. [Fuente](https://catalog.ihsn.org/index.php/catalog/5265/download/64548).

En la muestra hay factores de expansión para cada tipo de entidad (persona, hogar, viaje). Por ejemplo:

In [ ]:
tabla_completa[['FactorExpansion', 'FactorPersona']].describe()

Para calcular el peso o representatividad de un viaje debemos multiplicar de su factor de expansión (_qué tan frecuente ese tipo de viaje es_) y el del factor de expansión de cada persona (_qué tan representativa de otras personas es_).

Además hay factores de expansión para días de semana, sábado, domingo, y periodos estival (vacaciones) y normal. Trabajaremos con los periodos normales. Guardaremos el factor de expansión completo en la columna `Peso`.

In [ ]:
viajes.groupby('TipoDia')['FactorExpansion'].describe()

In [ ]:
tabla_completa["Peso"] = tabla_completa["FactorExpansion"] * tabla_completa["FactorPersona"]
tabla_completa.groupby('TipoDia')['Peso'].describe()

Ahora comencemos a responder preguntas.

## ¿Cuál es la distribución de uso de modo de transporte en viajes al trabajo?

Una pregunta relevante siempre, porque las ciudades cambian constantemente, En tiempos de COVID-19 estuvo en debate si el uso de transporte público es un foco de contagio. Entender la distribución de usos de transporte por comuna es importante para la definición de estrategias de desconfinamiento. En la actualidad, con una expansión urbana y un cambio en la composición poblacional (hay más migrantes, la población ha envejecido), los patrones de viaje no son necesariamente los de antes.

Aunque, cierto, los que tiene la encuesta son los oficiales. Así que necesitamos conocerlos de todos modos.

Debemos calcular la distribución de uso de modos de transporte por comuna, pero primero debemos preguntarnos: ¿cuáles viajes analizaremos? El título indica que nos interesan los viajes al trabajo, ya que son viajes recurrentes. 

¿Cuántos tipos de viaje hay?

In [ ]:
tabla_completa['Proposito'].value_counts(normalize=True)

In [ ]:
total_viajes = (
    tabla_completa.groupby("Proposito")["Peso"]
    .sum()
    .astype(int)
    .sort_values(ascending=False)
)

total_viajes / total_viajes.sum()

La columna `ModoDifusion` contiene el modo de transporte.

In [ ]:
modo_comuna = (
    # filtramos viajes al trabajo
    tabla_completa[tabla_completa["Proposito"] == "Al trabajo"]
    # una persona puede tener múltiples viajes al trabajo durante el día.
    # por ejemplo, cuando sale a almorzar y después vuelve.
    # ese viaje no es relevante para este análisis.
    .drop_duplicates(subset="Persona", keep="first")
    # agrupamos por comuna y modo de transporte
    .groupby(["Comuna", "ModoDifusion"])
    # sumamos los factores de expansión
    ["Peso"].sum()
    # convertimos a una matriz
    .unstack(fill_value=0)
)

modo_comuna

Veamos esta tabla como un gráfico que nos permita comparar la distribución por comunas. Usemos el método `plot` de `pandas`:

In [ ]:
modo_comuna.plot()

No es útil. No respeta lo que hemos visto en clases: utiliza líneas para conectar categorías, pero la pendiente no tiene significado.

In [ ]:
modo_comuna.plot(kind='bar', linewidth=0, stacked=True)

In [ ]:
modo_comuna.plot(kind='barh', linewidth=0, stacked=True)

En `chiricoca` tenemos un método `barchart` que hace esto mismo pero de una manera que no oculta el utilizar `matplotlib` (y que, por lo mismo, más adelante veremos que podemos combinar con otros gráficos):

In [ ]:
from chiricoca.tables import barchart

ax = barchart(
    modo_comuna,
    stacked=True,
    sort_categories=True,
    sort_items=True,
    horizontal=True
)

ax.set_title("Uso de Modo de Transporte en Viajes al Trabajo")
ax.set_ylabel("")
ax.set_xlabel("Cantidad de Viajes")

¡Es un gráfico interesante! Sin embargo está complejo, ya que cuesta diferenciar y comparar las distintas categorías. Creemos una categorización más sencilla que nos permita comparar mejor:

In [ ]:
tabla_completa["ModoAgregado"] = tabla_completa["ModoDifusion"].map(
    {
        "Taxi": "Taxi",
        "Bip! - Otros Privado": "Público",
        "Bip!": "Público",
        "Bip! - Otros Público": "Público",
        "Taxi Colectivo": "Taxi",
        "Bicicleta": "Activo",
        "Caminata": "Activo",
        "Auto": "Auto",
        "Otros": "Otros",
    }
)

modo_comuna = (
    tabla_completa[tabla_completa["Proposito"] == "Al trabajo"]
    .drop_duplicates(subset=["Persona"], keep="first")
    .groupby(["Comuna", "ModoAgregado"])
    ["Peso"].sum()
    .unstack(fill_value=0)
)

modo_comuna

In [ ]:
ax = barchart(
    modo_comuna,
    stacked=True,
    sort_categories=True,
    sort_items=True,
    horizontal=True
)

ax.set_title("Uso de Modo de Transporte en Viajes al Trabajo")
ax.set_ylabel("")
ax.set_xlabel("# de Viajes")

In [ ]:
ax = barchart(
    modo_comuna,
    stacked=True,
    normalize=True,
    sort_categories=True,
    sort_items=True,
    horizontal=True
)

ax.set_title("Uso de Modo de Transporte en Viajes al Trabajo")
ax.set_ylabel("")
ax.set_xlabel("Fracción de los Viajes")

In [ ]:
from chiricoca.base.weights import variance_stabilization

sns.heatmap(modo_comuna.pipe(variance_stabilization), center=0)

Verlo nos lleva a preguntarnos si existe una relación entre las propiedades de una comuna y su uso de transporte público.

Para ello calcularemos el ingreso promedio en cada comuna. Al igual que con los viajes, debemos utilizar el ingreso considerando los factores de expansión:

In [ ]:
ingreso_por_comuna = (
    hogares.groupby("Comuna")
    .apply(
        lambda x: (x["FactorHogar"] * x["IngresoHogar"]).sum() / x["FactorHogar"].sum()
    )
    .rename("ingreso")
    .astype(int)
)

ingreso_por_comuna.sort_values()


Ahora que tenemos esta serie, podemos hacer un cruce entre las dos tablas que hemos calculado. utilizamos la función `normalize_rows` para normalizar los valores de cada comuna, y así poder compararlas:

In [ ]:
from chiricoca.base.weights import normalize_rows

modo_comuna_ingreso = modo_comuna.pipe(normalize_rows).join(ingreso_por_comuna)
modo_comuna_ingreso


Para comparar el uso de transporte público y el ingreso poddemos utilizar un `scatterplot`:

In [ ]:
modo_comuna_ingreso.plot(x="ingreso", y="Público", kind="scatter")


Aunque nos gustaría saber cuál es la comuna que corresponde a cada punto, el gráfico no lo dice. Además el eje `x` utiliza una notación que nos impide apreciar los valores totales. Para ello podemos utilizar el método `scatterplot` en `chiricoca`, en conjunto con configuraciones de matplotlib:

In [ ]:
from chiricoca.tables import scatterplot

In [ ]:
ax = scatterplot(
    modo_comuna_ingreso,
    "ingreso",
    "Público",
    annotate=True,
    avoid_collisions=True,
    label_args=dict(fontsize="small"),
    scatter_args=dict(color="purple"),
)

ax.set_xlabel("Ingreso Promedio por Hogar")
ax.set_ylabel("Proporción de Uso de Transporte Público")
ax.set_title(
    "Relación entre Uso de Transporte Público e Ingreso por Comunas de RM (Fuente: EOD2012)"
)
ax.grid(alpha=0.5)
ax.ticklabel_format(style="plain")

Observamos que los tres grupos de uso de transporte público son: las comunas fuera del radio urbano (esquina inferior izquierda), que no son más ricas que el resto y no usan transantiago porque no llega a ellas; las comunas ricas (inferior derecha), que casi no usan transporte público a pesar de estar bien conectadas; y el resto, que presenta tasas variables de uso de transporte público.
  

Vimos en clase el _ternary plot_. Podemos intentar hacer uno, utilizando la biblioteca `python-ternary`. Todavía me falta hacer una función dentro de `chiricoca` que la utilice a cabalidad, pero aquí hay una primera versión:

In [ ]:
from chiricoca.tables.ternary import ternary_scatter

ternary_scatter(
    modo_comuna[["Activo", "Auto", "Público"]].pipe(normalize_rows),
    s=modo_comuna_ingreso["ingreso"].div(modo_comuna_ingreso["ingreso"].max()) * 1000,
)

El resultado es interesante, pero le falta trabajo: ¿cómo etiquetar los ejes para que sean interpretables? ¿Cómo evitar que las etiquetas de los puntos se intersecten? También hace falta incorporar una leyenda que permita saber qué significa el tamaño de los puntos.

## ¿Cuáles son las rutinas en la ciudad?

Para mejorar el funcionamiento de una ciudad es clave entender qué se hace en ella y cuándo.

Sabemos que el qué se hace está codificado en el atributo categórico `Proposito`. También sabemos que el atributo categórico `DiaAsig` se refiere al día que está asignado a la persona que responde la encuesta (al día de sus viajes).

In [ ]:
tabla_completa["DiaAsig"]

Definiremos una rutina como la distribución de viajes por tipo de propósito en cada unidad de análisis (en este caso, un día).

Utilizaremos operaciones `groupby` para calcular esa distribución para cada uno de los días de la semana, en periodo normal.

In [ ]:
tabla_completa['Proposito'].value_counts().plot(kind='barh')

In [ ]:
rutinas = (
    tabla_completa
    .groupby(["DiaAsig", "Proposito"])["Peso"]
    .sum()
    .unstack()
    .loc[["lunes", "martes", "miércoles", "jueves", "viernes", "sábado", "domingo"]]
    # eliminaremos los viajes de volver a casa y de tipo otra actividad porque no ayudan a caracterizar las rutinas diarias.
    .drop(["volver a casa", "Otra actividad (especifique)"], axis=1)
)

rutinas


¿Cómo visualizar esta tabla? Una manera directa es utilizar el método `plot` de pandas, que usará un `linechart`. Veamos como luce:

In [ ]:
rutinas.plot()


No se ve bonito, pero tampoco configuramos nada del gráfico, solamente lo ejecutamos para tener una noción de cómo se verían los datos. 

A pesar de que podríamos utilizar líneas, ya que la progresión de lunes a domingo es ordinal y puede ser interpolada, no tiene un significado relevante para nosotros de acuerdo a la definición de rutina. Además la cantidad de categorías en los datos hace difícil distinguir una línea de otra.

Podemos hacer la misma exploración, esta vez con un `barchart`:

Ahora bien, ¿buscamos es identificar patrones relativos? En tal caso, podemos probar con un gráfico normalizado:

In [ ]:
barchart(rutinas, stacked=True, normalize=True, sort_categories=True)


Al usar un gráfico relativo encontramos diferencias que antes no parecían tan notorias. Por ejemplo, en proporción, los viajes de recreación son más frecuentes los fines de semana que de lunes a viernes. En el gráfico absoluto se notaba un ligero incremento, pero quizás lo interesante es que, así como suben los de recreación, bajan mucho los demás. 

Lo mismo sucede con visitar a alguien e ir de compras.

El gráfico de barras podría ser suficiente si lo que queremos es determinar si hay diferencias entre las rutinas. Con esta última versión, sabemos que son diferentes, y tenemos una noción de cuáles son las diferencias.

Sin embargo, si nuestra tarea consistiese en identificar elementos específicos de las rutinas, como puede ser _conocer los valores exactos de la distribución_, o _agrupar actividades de acuerdo a su distribución en varios días_, entonces debemos buscar otra alternativa.

Exploremos como luce un heatmap en este caso:

In [ ]:
from chiricoca.tables.heatmap import heatmap

heatmap(rutinas)


Lo que hicimos en el gráfico de barras fue normalizar las columnas de la tabla. Podemos hacer lo mismo. Y luego trasponerla para facilitar la lectura. Quedaría así:

In [ ]:
from chiricoca.base.weights import normalize_columns

heatmap(rutinas.pipe(normalize_columns).T)


Observamos que este heatmap nos permite apreciar las variaciones diarias en la proporción. Si lo configuramos para que muestre más información y tenga mejor apariencia podría ser el gráfico final de la tarea:

In [ ]:
ax = heatmap(
    rutinas.pipe(normalize_columns).T,
    annot=True,
    fmt=".2f",
    linewidth=0.5,
)

ax.set_ylabel("")
ax.set_xlabel("")
ax.set_title("Rutinas Diarias en Santiago")


Ese gráfico ya está terminado: podemos ver patrones globales gracias a la escala de colores, y podemos comparar e identificar valores específicos gracias a las anotaciones.

Todavía nos falta poder agrupar las actividades (o filas de la matriz) de acuerdo a su similitud. Afortunadamente lo podemos lograr cambiando el método empleado: usar un `clustermap` en vez de un `heatmap`:

In [ ]:
ax = heatmap(
    rutinas.pipe(normalize_columns).T,
    annot=True,
    fmt=".2f",
    linewidth=0.5,
    cluster_rows=True
)

ax.set_ylabel("")
ax.set_xlabel("")
ax.set_title("Rutinas Diarias en Santiago")


## ¿Son diferentes las rutinas entre hombres y mujeres? (_lollipop_: un gráfico no visto)

Una primera aproximación es mirar la cantidad de viajes.

In [ ]:
from chiricoca.base.weights import weighted_mean

In [ ]:
tabla_completa['FactorPersona']

In [ ]:
personas.groupby("Sexo").apply(
    lambda x: x["Viajes"].value_counts().sort_index()
).unstack(fill_value=0).T.plot(kind="barh")

In [ ]:
personas['Viajes'].mean(), weighted_mean(personas, 'Viajes', 'FactorPersona')

In [ ]:
viajes_por_sexo = (
    personas.merge(hogares)
    .groupby(["Sexo", "Sector"])
    .apply(lambda x: weighted_mean(x, "Viajes", "FactorPersona"))
    .unstack()
    .T
)
viajes_por_sexo

In [ ]:
fig, ax = plt.subplots()

ax.plot(viajes_por_sexo['Hombre'], marker='o', linestyle='none')
ax.plot(viajes_por_sexo['Mujer'], marker='s', linestyle='none')
ax.vlines(x=viajes_por_sexo.index, ymin=viajes_por_sexo['Hombre'], ymax=viajes_por_sexo['Mujer'], color='grey')

In [ ]:
fig, ax = plt.subplots()

plot_df = viajes_por_sexo
plot_df = viajes_por_sexo.assign(diff=lambda x: x['Mujer'] - x['Hombre']).sort_values('diff')

plot_index = range(len(plot_df))

ax.scatter(x=plot_df['Hombre'], y=plot_index, marker='o', label='Hombres', color='orange', zorder=2)
ax.scatter(x=plot_df['Mujer'], y=plot_index, marker='s', label='Mujeres', color='green', zorder=2)
ax.hlines(y=plot_index, xmin=plot_df['Hombre'], xmax=plot_df['Mujer'], color='grey', zorder=1)

ax.set_yticks(plot_index, labels=plot_df.index)
ax.grid(linestyle='dotted', alpha=0.5, zorder=0)

ax.legend()

sns.despine(ax=ax, left=True)
ax.set_title('¿Cuántos viajes hacen hombres y mujeres?', loc='left')
ax.set_xlabel('Cantidad de viajes')

fig.tight_layout()

## ¿Qué hacen las personas durante el día?

Inspirándonos en el ejemplo de visualización del New York Times sobre cómo son los días de las personas, nos preguntamos: **¿Qué hacen las personas durante el día?** Podemos aproximarlo utilizando los datos de la encuesta. Para ello tendremos que calcular para cada minuto del día qué está haciendo cada persona y calcular la distribución por minuto.

Tenemos todo lo necesario: la hora de inicio del viaje `HoraIni`, su duración `TiempoViaje` (en minutos) y el `Proposito`. Necesitaremos lo siguiente de pandas:

* La clase `Timedelta` y las funciones `to_timedelta` y `timedelta_range` para ayudarnos a calcular lo que está haciendo una persona.
* `shift` para combinar una celda con la previa/siguiente, de modo de poder determinar las actividades.

Tendremos una versión incompleta y queda propuesto para ustedes completarla y extender su análisis.

Primero, es necesario construir una tabla de actividades. Una actividad es lo que sucede _después_ del viaje, es el motivo por el que se realiza un viaje. Como tenemos viajes, debemos construir las actividades a partir de ellos.

In [ ]:
tabla_completa.columns

In [ ]:
tabla_completa['TipoDia']

In [ ]:
from huedhued.time import time_matrix

matriz_de_tiempo = time_matrix(tabla_completa[tabla_completa['TipoDia'] == 'Laboral'])
matriz_de_tiempo

In [ ]:
heatmap(matriz_de_tiempo.T.pipe(normalize_columns), cluster_rows=True)

In [ ]:
df_datetime = matriz_de_tiempo.set_index(
    pd.to_datetime(matriz_de_tiempo.index.total_seconds(), unit="s")
)

df_datetime.index

In [ ]:
dia_base = df_datetime.index[0].date()
hora_inicio = pd.Timestamp(dia_base).replace(hour=5, minute=0)
hora_fin = pd.Timestamp(dia_base + pd.Timedelta(days=1)).replace(hour=0, minute=0)

In [ ]:
vis_df = df_datetime[
    (df_datetime.index >= hora_inicio) & (df_datetime.index <= hora_fin)
]

In [ ]:
from chiricoca.tables import streamgraph
import matplotlib.dates as mdates

ax = streamgraph(
    vis_df,
    baseline="zero",
    palette="husl",
    sort_areas="max",
    fig_args=dict(figsize=(8, 4)),
    legend=True,
)

def configurar_ax(ax):
    # Configurar formato de hora
    hora_fmt = mdates.DateFormatter("%H:%M")
    ax.xaxis.set_major_formatter(hora_fmt)

    # Especificar manualmente los ticks para asegurar que aparezca medianoche
    horas = list(range(5, 24)) + [0]  # 6 AM hasta medianoche (0)
    fechas_ticks = [pd.Timestamp(dia_base).replace(hour=h) for h in horas[:-1]] + [hora_fin]
    ax.set_xticks(fechas_ticks)

    ax.grid(True, linestyle="dotted", color="#efefef")
    ax.set_title("¿Qué hacen los y las santiaguinas en un día laboral?", loc="left")
    ax.set_ylabel("Proporción de las actividades")

configurar_ax(ax)

In [ ]:
ax = streamgraph(
    vis_df,
    normalize=True,
    baseline="zero",
    avoid_label_collisions=True,
    palette="flare",
    sort_areas="peak_time",
    area_args=dict(edgecolor="black", linewidth=0.25),
    fig_args=dict(figsize=(8, 4)),
    legend=True
)

configurar_ax(ax)

¡Es un buen comienzo! Sin embargo, todavía falta mucho por hacer. Algunas ideas:

- ¿Hay diferencias de género, edad y otros grupos?
- El gráfico no normalizado debiese tener siempre la misma cantidad de personas en cada instante del día. Entonces, ¿Cómo incluir a la gente que no realizó viajes? ¿Cómo incluir el antes y el después de los viajes? 

Las mejoras al gráfico de tipo estético y funcional las veremos en las siguientes clases.

## Sobre herramientas y Python

Con estas herramientas podemos explorar las relaciones que hay entre las variables de nuestro dataset. Concluimos que la mayor dificultad no está en _implementar_ las visualizaciones, sino en, primero, **saber qué preguntarle a los datos**, y segundo, **elegir los métodos adecuados para responder la pregunta**. Probablemente seaborn, pandas o matplotlib tienen dicha solución implementada, o al menos a unos pasos de ser implementada. También podemos utilizar los métodos implementados en `chiricoca`.

El siguiente paso es entender cómo se comportan estos métodos con otras variables del dataset. También hemos probado distintos valores para atributos de apariencia, como los tamaños de figura y las paletas de colores.

Una dificultad en el aprendizaje es que no existen estándares para nombrar a los métodos y sus parámetros. Por ejemplo, el parámetro de la paleta de colores se suele llamar `cmap` en `matplotlib` y `pandas`, pero se llama `palette` en casi todos los métodos de `seaborn` --- digo casi todos porque algunos también usan `cmap`. Esto puede ser confuso para aprender, y creo que de momento no hay una solución más que ejercitar y aprenderse los nombres de parámetros y de métodos que sean más adecuados para la tarea a resolver.